# Submesoscale processes in the ice-impacted SO

This notebook describes the analysis followed by Giddy et al. 

To start, data is called and plotted as an overview

* Linearly Interpolated Seaglider data are input (the pre-processing and interpolation will be provided in a separate notebook)
* Additional data is called from ERA5, Copernicus, Bremen, AVISO
* Some regional plots are produced - these feed into Figure 1

Then, the following analyses are performed:

* Analysis of upper ocean vertical and horizontal buoyancy
* Turner angle analysis to assess the contributions of salinity and temperature to vertical and horizontal stratification
* MLI growthrates
* Atmospheric derived equivalent heat fluxes are computed (Qnet,FWF,mesoscale EBF)
* Submesocale equivalent heat fluxes are calculated, an example of the methodology is provided
* Spectral Analysis of the mixed layer density variation 
* Spice analysis to explain cross isopycnal separation 


In [1]:
# Load modules

import numpy as np
import os, sys
sys.path.insert(0, 'functions/')

import sms_fluxes
import gsw

import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import cmocean.cm as cmo

import glidertools as gt

import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes ,InsetPosition,mark_inset
import cmocean.cm as cmo

from scipy.interpolate import griddata as g


/Users/isabelle/opt/anaconda3/lib/python3.7/site-packages/glidertools/mapping.py:1336: GliderToolsWarning: PyKrige is not installed. To enable the variogram function please run `pip install pykrige`. Variograms are required for sensible 2D interpolation.
  warnings.warn(message, category=GliderToolsWarning)


In [2]:
from matplotlib import rc
rc('font',**{'size':14})
params = {'mathtext.default': 'regular' }          
plt.rcParams.update(params)

In [3]:
# Load data

In [4]:
# Glider data - sg643
# This data has already been interpolated
datadir ='data/sg643/'
dat=xr.open_dataset(os.path.join(datadir,'sg643_linterp.nc'))


SA=(dat.salt)
CT=(dat.temp)
rho=(dat.dens)
depth=dat.depth
lat=(dat.lat[15,:])
lon=(dat.lon[15,:])
dist=(dat.distance)
glider_dir=np.array(sms_fluxes.calc_glider_traj(lat,lon)) # Glider Trajectory

In [5]:
time=pd.Series(dat.time[20:25,:].mean(dim='depth')).interpolate(method='pad')

In [6]:
# Load auxillary data

In [7]:
# ERA5

# Winds - ERA5
era5_winds=xr.open_dataset('data/era5/era5_winds.nc')
era5w_subset=era5_winds.sel(latitude=-60,longitude=0)

#Heat fluxes
# Load annual cycle heat flux
era5_annual = xr.open_dataset('data/era5/era5_htflux.nc')
latent=era5_annual.mslhf.mean(dim='latitude').mean(dim='longitude')
longwave=era5_annual.msnlwrf.mean(dim='latitude').mean(dim='longitude')
shortwave=era5_annual.msnswrf.mean(dim='latitude').mean(dim='longitude')
sensible=era5_annual.msshf.mean(dim='latitude').mean(dim='longitude')
Qnet = latent+longwave+shortwave+sensible

#Freshwater Fluxes
erafw = xr.open_dataset('data/era5/sg643_fwf.nc')
erafw=erafw.sel(latitude=-60.0,longitude=0)
fwfms = (erafw.mer+erafw.mtpr)/3600  # convert to seconds


In [8]:
xr.open_dataset?

In [10]:
# Sea Ice

datadir='data/'
ds_si=xr.open_mfdataset(os.path.join(datadir,'seaice/*.nc'),combine='nested',concat_dim='time')
ds_coords=xr.open_dataset(os.path.join(datadir,'seaice/LongitudeLatitudeGrid-s6250-Antarctic.hdf'))
ln = ds_coords.Longitudes.values
lt = ds_coords.Latitudes.values



In [7]:

ls = os.listdir(os.path.join(datadir,'seaice'))
ls.sort()
sic_date=[]
for filename in ls:
    if filename[-2:] == 'nc':
        try:
            sic_date.append(pd.to_datetime(str(filename)[-14:-5]))
        except: 'Value Error', sic_date.append(pd.to_datetime(str(filename)[-16:-8]))
sic_date=pd.Series(sic_date)

seaice=ds_si.ASI_Ice_Concentration

print('starting interpolation')
X = np.arange(-180 ,180, 0.05)
Y = np.arange(-80, -49.95, 0.05)
x, y = np.meshgrid(X,Y)
sic_new = np.ndarray([len(seaice.time), len(Y), len(X)])
print(sic_date[348])

i=348
print('ravelling')
x_, y_ = np.ravel(ln), np.ravel(lt)
x_[x_>180] = x_[x_>180]-360
print('ravelling sea ice')
si = np.ravel(seaice[i,:,:])
print(si.shape)
print('doing the interpolation')
sic_new[i,:,:] = g((y_, x_), si, (y, x), method='linear')
sic_new[sic_new==0] = np.NaN

# identify indices of seaice at roammiz site
ind=np.array(np.where((lt>-61)&(lt<-59)&(ln>-1)&(ln<1)))


OSError: [Errno -51] NetCDF: Unknown file format: b'/Users/isabelle/Documents/git/roammiz-giddy-2020/data/seaice/LongitudeLatitudeGrid-s6250-Antarctic.hdf'

In [ ]:
# Sentinal 3A, Chl sat OCLI
chl=xr.open_dataset('data/chla/S3A_OL_2_WFR____20190107T090544_20190107T090844_20190107T105111_0179_040_064_3960_MAR_O_NR_002.nc')

In [ ]:
# AVISO

avisoAn=xr.open_dataset('data/aviso/aviso_annual.nc')
aviso2=xr.open_dataset('data/aviso/aviso_2018_2019.nc')

#avisoAn=avisoAn.sel(latitude=-60.125,longitude=360.125)
adt=aviso2.sel(latitude=slice(-65,-55),longitude=slice(-5,5))

In [ ]:
# SMOS satellite Sea Surface Salinity
smos = xr.open_mfdataset('data/smos/SMOS*.nc')

## Regional Setting

In [ ]:
### SMOS Figure

from obspy.imaging.cm import viridis_white
import seaborn as sns
import cartopy
import cartopy.crs as ccrs

import matplotlib
import matplotlib.patches as mpatches
from cartopy.util import add_cyclic_point
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import shapefile   
from mpl_toolkits.axes_grid1.inset_locator import inset_axes ,InsetPosition,mark_inset

import matplotlib.path as mpath
import cmocean.cm as cmo

sns.set(font_scale=2)

# Load sea ice extent poplygons

fname1 = r'data/seaice/seaice_extent/extent_S_201809_polyline_v3.0.'
fname2 = r'data/seaice/seaice_extent/extent_S_201902_polyline_v3.0.'

projection =ccrs.SouthPolarStereo(central_longitude=0) # set projection

fig, ax = plt.subplots(1, figsize=(15,15),
                       subplot_kw={'projection': projection})

# SMOS SSS data
cs=smos.sel(time='2018-12-16',lat=slice(-83.517136,-40.131504)).SSS.plot.pcolormesh(ax=ax,transform=ccrs.PlateCarree(),vmin=33.4,vmax=34.6,cmap=viridis_white,add_colorbar=False)
cax = fig.add_axes([0.45, 0.05, 0.4, 0.05])
cbar=plt.colorbar(cs,cax=cax,orientation='horizontal',format='%.1f',shrink=0.5)
cbar.set_label('SMOS SSS (g kg$^{-1}$)')


#Ice
cf=ax.pcolormesh(X,Y,sic_new[348,:,:], transform=ccrs.PlateCarree(),cmap='Greys')
cax = fig.add_axes([0.01, 0.05, 0.4, 0.05])
cbar2=plt.colorbar(cf,cax=cax,orientation='horizontal',format='%.0f',shrink=0.5)
cbar2.set_label('Sea Ice Concentration')

#Sea Ice extent 
# Max Extent 2018
sf = shapefile.Reader(fname1)
for shape in sf.shapeRecords():
    for i in range(len(shape.shape.parts)):
        i_start = shape.shape.parts[i]
        if i==len(shape.shape.parts)-1:
            i_end = len(shape.shape.points)
        else:
            i_end = shape.shape.parts[i+1]
        listx = [i[0] for i in shape.shape.points[i_start:i_end]]
        listy = [i[1] for i in shape.shape.points[i_start:i_end]]
        ax.plot(listx,listy,transform=ccrs.SouthPolarStereo(),c='orange', linestyle='--',linewidth=4,label='Max Sea Ice Extent')

# Min Extent 2019
sf = shapefile.Reader(fname2)
for shape in sf.shapeRecords():
    for i in range(len(shape.shape.parts)):
        i_start = shape.shape.parts[i]
        if i==len(shape.shape.parts)-1:
            i_end = len(shape.shape.points)
        else:
            i_end = shape.shape.parts[i+1]
        listx = [i[0] for i in shape.shape.points[i_start:i_end]]
        listy = [i[1] for i in shape.shape.points[i_start:i_end]]
        ax.plot(listx,listy,transform=ccrs.SouthPolarStereo(),c='orange',linestyle='-',linewidth=4,label='Min Sea Ice Extent')


# Study site
ax.add_patch(mpatches.Rectangle(xy=[-5, -64], width=10, height=10,
                                    facecolor='None',edgecolor='tab:red',linewidth=8,
                                    alpha=1,
                                    transform=ccrs.PlateCarree()))

ax.scatter(0,-60,c='red',s=60,marker='D', transform=ccrs.PlateCarree())


# Some edits to the plot
ax.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='black', facecolor='Grey')

theta = np.linspace(0, 2*np.pi, 100)   # Make plot a circle
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

ax.set_boundary(circle, transform=ax.transAxes)
ax.outline_patch.set_visible(False)

#plt.savefig('figures/map2018_smos_v6.png',bbox_inches='tight',transparent=True,dpi=100)

In [ ]:
## Figure 1b - Chl
### Chlorophyll

import matplotlib
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
from matplotlib import ticker, cm
from matplotlib.colors import LogNorm

from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica'],'size':22})
params = {'mathtext.default': 'regular' }          
plt.rcParams.update(params)


projection =ccrs.PlateCarree()#
data_crs=ccrs.PlateCarree()

fig, ax = plt.subplots(1, 1, figsize=(15,15),
                       subplot_kw={'projection': projection})

ax.set_extent([-4, 2, -62, -56])

# Chl data
chlnorm=chl.CHL_OC4ME
z = np.ma.masked_where(chlnorm <= 0, chlnorm)
cs=ax.pcolormesh(chl.longitude,chl.latitude,z,vmin=0, vmax=1,cmap=cmo.delta,transform=data_crs)

cax,kw = matplotlib.colorbar.make_axes(ax,location='top',pad=0.01,shrink=0.7)
out=fig.colorbar(cs,cax=cax,**kw)
label=out.set_label('Chl (mg m$^{-3}$)',size=22,verticalalignment='bottom')

# Seaglider location
ax.scatter(lon[522:644],lat[522:644],facecolor='tab:red',s=12,transform=data_crs)

# Location of the Southern Boundary of the ACC
ax.contour(adt.longitude,adt.latitude,adt.adt[38,:,:], levels=[-1.244],colors='magenta',linestyles='--',transform=data_crs,linewidths=4)

# Plot cleaning
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0, linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlocator = mticker.FixedLocator([-4, -2, 0, 2,4])
gl.ylocator = mticker.FixedLocator([-62, -60, -58, -56,-54])
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 28, 'color': 'k'}
gl.ylabel_style = {'size': 28, 'color': 'k'}

#plt.savefig('figures/chl_summer_2018_v3.png',transparent=True,dpi=100,bbox_inches='tight')


In [ ]:
# Seasonality

import seaborn as sns
fig,ax=plt.subplots(1,1,figsize=[26,8],sharex=True)
sns.set(font_scale=3)


sns.set_style('white')
#sns.set_style("ticks", {"xtick.major.size": 20, "ytick.major.size": 20})
sns.set(font_scale=3)
sns.set_style("ticks", {"xtick.major.size": 20, "ytick.major.size": 20})


#heat flux
ax.plot(era5_annual.time,pd.Series(Qnet).rolling(24).mean(),c='tab:red',linewidth=3,alpha=0.8)
ax.set_ylabel('Heat Flux (W/m$^2$)',c='tab:red')
ax.tick_params(axis='y', colors='tab:red')

#seaice
ax3=ax.twinx()
ax3.plot(sic_date,np.nanmean(np.nanmean(seaice[:,ind[0],ind[1]],axis=1),axis=1),c='teal',linewidth=3,alpha=0.8)
ax3.set_ylabel('Sea Ice Concentration',c='teal')

ax3.tick_params(axis='y', colors='teal')

ax3.set_xlim(np.array(avisoAn.time[30]),np.array(avisoAn.time[-1]))


ax3.axvline(np.array(avisoAn.time[258]),c='Grey',linestyle='--',alpha=0.7) # Heat flux positive
ax3.axvline(np.array(avisoAn.time[359]),c='Grey',linestyle='--',alpha=0.7) # Heat flux negative


# Mark start and end of deployment
ax3.annotate('Deploy Glider', xy=(np.array(avisoAn.time[258]),20),  xycoords='data',
            xytext=(np.array(avisoAn.time[265]),40),fontsize=27,
            arrowprops=dict(arrowstyle="fancy",
                            fc="0.1", ec="none",
                            connectionstyle="angle3,angleA=0,angleB=-90"))

ax3.annotate('Retrieve Glider', xy=(np.array(avisoAn.time[359]),10),  xycoords='data',
            xytext=(np.array(avisoAn.time[320]),30),fontsize=27,
            arrowprops=dict(arrowstyle="fancy",
                            fc="0.1", ec="none",
                            connectionstyle="angle3,angleA=0,angleB=-90"))

# Plot formatting
months = mdates.MonthLocator(interval=2, tz=None)
ax.xaxis.set_major_locator(months)
h_fmt= mdates.DateFormatter('%b %Y')
ax.xaxis.set_major_formatter(h_fmt)

#plt.savefig('figures/sg643_fig1_seasonality.png',transparent=True,dpi=75,bbox_inches='tight')


## Vertical and Horizontal Buoyancy

In [ ]:
## Put this somewhere

# Distance of glider sample site from Southern Boundary
import gsw as gsw
gsw.distance([0,0],[-60,-56])/1000

In [ ]:
lat_point=-60
f = sms_fluxes.coriolis_acc(np.nanmean(lat_point)) # Coriolis Acceleration

alpha,beta= sms_fluxes.alphabeta(SA,CT,depth) #Thermal Expansion Coeffient and Saline Contraction Coefficient

mld = pd.Series(sms_fluxes.calc_mld(np.array(rho),np.array(depth),ref_depth=10))  #Mixed Layer Depth

Buoyancy (kindof the inverse of density) is defined as

$$
b _x = g(1-\rho/\rho_0)
$$



In [ ]:
# Buoyancy including salinity and temperature contributions

g=9.8
by=(9.8*(1-(rho[:]/1027)))
b=g*alpha[:,:-1]*np.diff(CT,axis=1)-g*beta[:,:-1]*np.diff(SA,axis=1)
bT=g*alpha[:,:-1]*CT.diff(dim='distance')/1000   # Temperature contribution
bS=-g*beta[:,:-1]*SA.diff(dim='distance')/1000    # Salinity contribution
bgrad=np.diff(by)/1000

# average lateral buoyancy gradient in the mixed layer (at least 15m above the ML)
bxml=np.ndarray(len(rho[1,:-1]))
for i in range(len(rho[1,:-1])):
    bxml[i]=(np.nanmean(bgrad[:np.int8(mld[i])-15,i],0))
    
bxml=pd.Series(bxml).interpolate()

# point lateral buoyancy gradient in the middle of the ML
bxmd=np.ndarray(len(rho[1,:-1]))
for i in range(len(rho[1,:-1])):
    bxmd[i]=bgrad[np.int8(mld[i]/2),i]
    
bxmd=pd.Series(bxmd).interpolate()

In [ ]:
# Difference between ML buoyancy gradient choices

plt.figure(figsize=[15,4])
plt.plot(np.abs(bgrad[15,:]),label='b15')
plt.plot(np.abs(bxmd[:]),alpha=0.6,label='b_middle')
plt.plot(np.abs(bxml),alpha=0.6,label='b_avg')
plt.legend()
#plt.savefig('figures/bx_comparison.png',bbox_inches='tight',dpi=75)


Vertical stratification is defined by the Brunt Vaisala frequency

$$
N^2 = \delta b / \delta z 
$$

It can be separated into temperature and salinity components

$$
N^2_T = -g\alpha \frac {\delta T} {\delta z} 
$$

$$
N^2_S = N^2 - N^2_T
$$

In [ ]:
# Vertical Stratification

import gsw
import pandas as pd
_,y=np.meshgrid(dist,depth)

n2 = np.diff(by*-1,axis=0)
n2_t=(-g*alpha[:-1,:]*np.diff(CT,axis=0))   # Temperature component
n2_s = n2-n2_t # Salt component

# ML N2
# Total
n2ml=np.ndarray(len(rho[1,:-1]))
for i in range(len(rho[1,:-1])):
    n2ml[i]=(np.nanmean(n2[(np.int8(mld[i])-5):np.int8(mld[i])+5,i],0))
    
n2ml=pd.Series(n2ml).interpolate()

# Temperature component
n2mlT=np.ndarray(len(rho[1,:-1]))
for i in range(len(rho[1,:-1])):
    n2mlT[i]=(np.nanmean(n2_t[np.int8(mld[i])-5:np.int8(mld[i])+5,i],0))
    
n2mlT=pd.Series(n2mlT).interpolate()

# Salinity component
n2mlS=np.ndarray(len(rho[1,:-1]))
for i in range(len(rho[1,:-1])):
    n2mlS[i]=(np.nanmean(n2_s[np.int8(mld[i])-5:np.int8(mld[i])+5,i],0))
    
n2mlS=pd.Series(n2mlS).interpolate()


In [ ]:
import seaborn as sns
sns.set(font_scale=2)
sns.set_style("whitegrid")
sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})

from matplotlib import pyplot, lines
from matplotlib import ticker
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True) 
formatter.set_powerlimits((-1,1)) 

fig,ax=plt.subplots(5,1,figsize=[18,16],sharex=True,constrained_layout=True)

ax[0].set_title('Absolute Salinity')
x,y=np.meshgrid(dist,depth)
cs=ax[0].pcolormesh(x,y,SA,cmap=cmo.haline,vmin=33.9,vmax=34.7)
ax[0].contour(x[100:,:],y[100:,:], CT[100:,:],levels=[0],colors='magenta',linewidths=2,alpha=0.7) # Bottom of WW layer defined as CT=0

cbar=plt.colorbar(cs,ax=ax[0],aspect=10)
cbar.set_label('(g kg$^{-1}$)')
levels=np.arange(1027,1028,0.1)
ax[0].contour(x,y, rho,levels=levels,colors='k',linewidths=0.8,alpha=0.7)
ax[0].plot(dist[:],pd.Series(mld).rolling(3).median(),c='w',linewidth=2)

ax[0].set_ylim(150,0)
ax[0].set_ylabel('Depth (m)')

ax[1].set_title('Buoyancy')
x,y=np.meshgrid(dist,depth)
cs=ax[1].pcolormesh(x,y,by,cmap=cmo.balance,vmin=-0.007,vmax=-0.001)
cbar=plt.colorbar(cs,ax=ax[1],aspect=10,pad=0.001)
cbar.formatter.set_powerlimits((0, 0))
cbar.formatter._useMathText = True


cbar.update_ticks()
cbar.set_label('(m s$^{-2})$')
levels=np.arange(1027,1028,0.1)
ax[1].contour(x,y, rho,levels=levels,colors='k',linewidths=0.8,alpha=0.7)
ax[1].plot(dist[:],pd.Series(mld).rolling(3).median(),c='w',linewidth=2)
ax[1].contour(x[100:,:],y[100:,:], CT[100:,:],levels=[0],colors='magenta',linewidths=2,alpha=0.7)

ax[1].set_ylabel('Depth (m)')


ax[1].set_ylim(150,0)

ax[2].set_title('10m buoyancy')

ax[2].plot(dist[:-1],np.array(by[10,0])+np.nancumsum(bgrad[10,:])*1000,c='k',label=u'$b$')#'\\\rho_0$')
ax[2].plot(dist[:-1],np.array(by[10,0])+np.nancumsum(bT[10,:])*1000,c='tab:orange',label=u'$b_T$')
ax[2].plot(dist[:-1],np.array(by[10,0])+np.nancumsum(bS[10,:])*1000,c='tab:blue',label=u'$b_S$')


ax[2].legend(loc='best',framealpha=0.5,fontsize=14)
formatter = ticker.ScalarFormatter(useMathText=True)

ax[2].ticklabel_format(axis='y', style='sci', scilimits=(-3,3))
ax[2].yaxis.major.formatter._useMathText = True

ax[2].set_ylabel('b (m $s^{-2})$')

ax[3].set_title('Mixed Layer Lateral Buoyancy Gradient')
ax[3].axhline(0.5e-7, c='Gray',linewidth=2)
ax[3].plot(dist[:-1],np.abs(bxml),c='k')

ax[3].set_ylabel('$b_x (s^{-2})$')
ax[3].ticklabel_format(axis='y', style='sci', scilimits=(-7,-7))
ax[3].yaxis.major.formatter._useMathText = True

ax[4].set_title('Brunt Vaisala Frequency')
ax[4].plot(dist[:-1],pd.Series(n2ml).rolling(6).median(),c='k',label='$\overline{N}{^2}(T,S)_{ML}$')
ax[4].plot(dist[:-1],pd.Series(n2mlT).rolling(6).median(),c='tab:orange',label='$\overline{N}{^2}(S)_{ML}$')
ax[4].plot(dist[:-1],pd.Series(n2mlS).rolling(6).median(),c='tab:blue',label='$\overline{N}{^2}(T)_{ML}$')

ax[4].axhline(0,c='Grey',alpha=0.5,linestyle='--')
ax[4].legend(loc='best',framealpha=0.5,fontsize=14)
ax[4].set_ylim(-1e-5,4e-4)
ax[4].set_ylabel('$N^2$ $(s^{-1})$')
ax[4].set_xticks([0,250000, 500000,750000, 1000000,1250000,1500000,1750000,2000000])  # this is a bit of a hack 
ax[4].set_xticklabels(['15 Dec','26 Dec','06 Jan','17 Jan','28 Jan',
                       '10 Feb','23 Feb','08 Feb','22 Feb'])

ax[4].set_xlabel('Time')

ax[4].ticklabel_format(axis='y',style='sci',scilimits=(-1,1))
ax[4].yaxis.major.formatter._useMathText = True
ax[4].set_xlim(dist[0],dist[-1])


# Add lines to show mesoscale transects
x,y = np.array([[0, dist[360]], [-5, -5]])
line = lines.Line2D(x, y, lw=5., color='r', alpha=0.9)
ax[0].add_line(line)
line.set_clip_on(False)

x,y = np.array([[dist[1280], dist[1550]], [-5, -5]])
line = lines.Line2D(x, y, lw=5., color='r', alpha=0.9)
ax[0].add_line(line)
line.set_clip_on(False)

ax[0].text(dist[120],-10,'N-S transect',fontsize=16)
ax[0].text(dist[1320],-10,'N-S transect',fontsize=16)


#plt.savefig('figures/buoyancy_summer_2018_v5.png',transparent=True,dpi=75,bbox_inches='tight')



## Turner Angle Analysis

The verticle turner angle defined as:

$$
Tu = \arctan(R_p)
$$

where 
$$
Rp = \alpha \frac {\delta T} {\delta z} (\beta \frac {\delta S} {\delta z})^{-1}
$$


Similarly, the horizontal turner angle is defined by replacing the vertical gradient with the horizontal:

$$
Rp_{horizontal} = \alpha \frac {\delta T} {\delta x} (\beta \frac {\delta S} {\delta x})^{-1}
$$







In [ ]:
# vertical
Rpx=(alpha[:-1,:]*(np.diff(CT,axis=0))/(beta[:-1,:]*(np.diff(SA,axis=0))))
Tux=(np.arctan(Rpx))

# Lateral
RpxL=(alpha[:,:-1]*(np.diff(CT,axis=1)/1000)/(beta[:,:-1]*(np.diff(SA,axis=1)/1000)))
TuxL=pd.Series((np.arctan(RpxL[15,:]))).interpolate()

In [ ]:
import seaborn as sns
sns.set(font_scale=1.5)

sns.set_style('white')
sns.set_color_codes()


bins = np.linspace(-np.pi/2,np.pi/2, 20)

fig,ax=plt.subplots(1,2,figsize=[15,7])
ax[0].set_title('Vertical Tu')


summer_subset=pd.Series(np.nanmean(Tux[10:200,10:490],axis=1)).interpolate()
sample = np.random.choice(summer_subset, size=300) 


lsummer_subset=pd.Series(np.nanmean(Tux[10:200,1467:1857],axis=1)).interpolate()
sample2 = np.random.choice(lsummer_subset, size=300) 


# Added confidence interval using bootstrapping
for _ in range(500):  #so B=10000
    sample_n = np.random.choice(summer_subset, size=300)
    sample_n2 = np.random.choice(lsummer_subset, size=300)


    sns.distplot(pd.Series((sample_n)), hist=False, bins=bins,rug=False,kde_kws={"color": "lightblue", "lw": 1,'alpha':0.2},ax=ax[0])
    sns.distplot(pd.Series((sample_n2)), hist=False,bins=bins, rug=False,kde_kws={"color": "pink", "lw": 1,'alpha':0.2},ax=ax[0])

sns.distplot(pd.Series(np.nanmean(Tux[10:200,:],axis=1)).interpolate(), hist=False,bins=bins ,rug=False,label='Total',kde_kws={"color": "k", "lw": 3},ax=ax[0])

sns.distplot(pd.Series(np.nanmean(Tux[10:200,10:490],axis=1)).interpolate(), hist=False, bins=bins,rug=False,label='Early Summer',kde_kws={"color": "b", "lw": 3},ax=ax[0])
sns.distplot(pd.Series(np.nanmean(Tux[10:200,1467:1857],axis=1)).interpolate(), hist=False,bins=bins, rug=False,label='Late Summer',kde_kws={"color": "r", "lw": 3},ax=ax[0])
ax[0].axvline(x=-np.pi/4,c='Gray',linestyle='--')
ax[0].axvline(x=np.pi/4,c='Gray',linestyle='--')
ax[0].axvline(x=0,c='Gray',linestyle='--')

ax[0].set_xticks([-np.pi/2, -np.pi/4, 0, np.pi/4,np.pi/2])
ax[0].set_xticklabels([r'- $\frac{\pi}{2}$',r'- $\frac{\pi}{4}$','$0$', r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$'])
ax[0].set_ylabel('Frequency')
ax[0].set_xlabel('Turner Angle')
ax[0].set_xlim(-np.pi/2,np.pi/2)

ax[1].set_title('Horizontal Tu')

bins = np.linspace(-np.pi/2,np.pi/2,20)


summer_subset=TuxL[10:390]
sample = np.random.choice(summer_subset, size=300) 

lsummer_subset=TuxL[1467:1857]
sample2 = np.random.choice(lsummer_subset, size=300) 


for _ in range(500):  #so B=10000
    sample_n = np.random.choice(summer_subset, size=300)
    sample_n2 = np.random.choice(lsummer_subset, size=300)


    sns.distplot(pd.Series((sample_n)), hist=False, bins=bins,rug=False,kde_kws={"color": "lightblue", "lw": 1,'alpha':0.2},ax=ax[1])
    sns.distplot(pd.Series((sample_n2)), hist=False,bins=bins, rug=False,kde_kws={"color": "pink", "lw": 1,'alpha':0.2},ax=ax[1])

sns.distplot(pd.Series((summer_subset)), hist=False, bins=bins,rug=False,label='Early Summer',kde_kws={"color": "b", "lw": 3},ax=ax[1])
sns.distplot(pd.Series((lsummer_subset)), hist=False,bins=bins, rug=False,label='Late Summer',kde_kws={"color": "r", "lw": 3},ax=ax[1])


sns.distplot(pd.Series((TuxL[:])), hist=False,bins=bins, rug=False,label='Total',kde_kws={"color": "k", "lw": 3},ax=ax[1])


ax[1].axvline(x=-np.pi/4,c='Gray',linestyle='--')
ax[1].axvline(x=np.pi/4,c='Gray',linestyle='--')
ax[1].axvline(x=0,c='Gray',linestyle='--')
ax[1].set_xticks([-np.pi/2, -np.pi/4, 0, np.pi/4,np.pi/2])            
ax[1].set_xticklabels(
           [r'- $\frac{\pi}{2}$',r'- $\frac{\pi}{4}$','$0$', r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$'])
ax[1].set_xlabel('Turner Angle')
ax[1].set_ylabel('Frequency')
ax[1].set_xlim(-np.pi/2,np.pi/2)

#plt.savefig('figures/Tu_summer_2018.png',transparent=True,dpi=75,bbox_inches='tight')

## ABI Growth Rate Analysis


Richardson number - gives idea of isopycnal slope steepness

$$
Ri = \frac {N^2} {(\delta u / \delta z)^2} \equiv \frac {f^2 N^2} {b_x^2}
$$

The fastest lengthscale and  growth rate following Stone 1970; Fox-Kemper et al. 2008, is defined as:

$$
l_{max} = \frac {2 \pi U} {f} \sqrt {\frac {1+Ri} {5/2}}$$  where 

$$
U \equiv \frac {|b_x|} f H
$$


$$
\tau_{max} = \sqrt {\frac {54} 5} \frac {\sqrt  {1+Ri}} {f}
$$


In [ ]:
# Richardson number
Ri = (1.26e-4**2*np.abs(n2[:,:-1]))/np.abs(bgrad[:-1,:])**2

#Ri in the ML
riml=np.ndarray(len(rho[1,:-1]))
for i in range(len(rho[1,:-1])):
    riml[i]=(np.nanmean(Ri[:np.int8(mld[i])-15,i],0))
riml[1705:1780]=np.nan
riml[-10:]=np.nan
riml[650:700]=np.nan
riml[1250:1500]=np.nan

logRi=np.log(np.abs(riml))
Ri_adj=logRi[logRi!=np.inf] #adjusted to exclude nans and infs


In [ ]:
# Length scale 
L1=(2*np.pi*mld[:-1]*np.abs(bxml)/f)/f*np.sqrt((1+np.abs(riml))/(5/2))
L=(2*np.pi*40*(10e-7)/f)/f*np.sqrt((1+np.abs(1))/(5/2))

# Wavelength with Ri=1
Lfk=(2*np.pi*100*np.abs(2*f**2)/f)/f*np.sqrt((1+np.abs(1))/(5/2))
print('Wavenumber:',Lfk/1000)

# Growth rate
Time= np.sqrt(54/5)*np.sqrt(1+abs(riml))/np.abs(f)
Time=Time[Time!=np.inf]

# Growth rate with Ri = 1
Time2= np.sqrt(54/5)*np.sqrt(1+1)/np.abs(f)
print('Growthrate:',Time2/3600)


In [ ]:
# Compare to BFK 2008
#h ~ 100 m
# bx ~ 2f^2
2*(f**2)

In [ ]:
fig,ax=plt.subplots(1,5,figsize=[18,6],sharey=True)
plt.subplots_adjust(wspace=0.1)
import numpy
import seaborn as sns

sns.set_style('white')
sns.set(font_scale=1.5)


# Subplot 1 - MLD
sns.distplot(mld[mld!=np.nan],bins=25,hist=True,norm_hist=False,rug=False,label='Total',kde=False,ax=ax[0])
ax[0].set_xlabel('MLD (m)')
ax[0].set_ylabel('Count')

axin0 = ax[0].inset_axes(
        [35, 550, 35, 250], transform=ax[0].transData)

sns.distplot(mld[mld!=np.nan][10:490],bins=25,hist=False,norm_hist=False,rug=False,kde=True,ax=axin0,kde_kws={'color':'b'})
sns.distplot(mld[mld!=np.nan][1450:1950],bins=25,hist=False,norm_hist=False,rug=False,kde=True,ax=axin0,kde_kws={'color':'r'})
axin0.set_xticks([])
axin0.set_yticks([])


# Subplot 2 - Buoyancy Gradient
sns.distplot(np.abs(bxml) ,bins=25,hist=True,norm_hist=False,rug=False,label='Total',kde=False,ax=ax[1])
ax[1].set_xlabel('|b$_x$| ($s^{-2}$)')

axin1 = ax[1].inset_axes(
        [0.9e-7, 550, 1.4e-7, 250], transform=ax[1].transData)

sns.distplot(np.abs(bxml)[10:490],bins=25,hist=False,norm_hist=False,rug=False,kde=True,ax=axin1,kde_kws={'color':'b'})
sns.distplot(np.abs(bxml)[1450:1950],bins=25,hist=False,norm_hist=False,rug=False,kde=True,ax=axin1,kde_kws={'color':'r'})
axin1.set_xticks([])
axin1.set_yticks([])

# Subplot 3 - Richardsn Number
sns.distplot(Ri_adj, hist=True,bins=25,rug=False,label='Total',kde=False,ax=ax[2],norm_hist=False)
ax[2].set_xlabel('Ri')
ax[2].set_xticks([0,5,10,15])
                
ax[2].set_xticklabels(
           [r'$10^{0}$',r'$10^5$','$10^{10}$', r'$10^{15}$'])


axin2 = ax[2].inset_axes(
        [5, 550, 12, 250], transform=ax[2].transData)


sns.distplot(Ri_adj[10:490],bins=25,hist=False,norm_hist=False,rug=False,kde=True,ax=axin2,kde_kws={'color':'b'})
sns.distplot(Ri_adj[1450:1950],bins=25,hist=False,norm_hist=False,rug=False,kde=True,ax=axin2,kde_kws={'color':'r'})
axin2.set_xticks([])
axin2.set_yticks([])


# Subplot 4 - Wavelength
sns.distplot(L1[L1<50000]/1000, hist=True,bins=25,norm_hist=False,rug=False,label='Total',kde=False,ax=ax[3])
ax[3].set_xlabel('Wavelength (km)')


axin3 = ax[3].inset_axes(
        [20, 550, 28, 250], transform=ax[3].transData)

sns.distplot((L1[L1<50000]/1000)[10:490],bins=25,hist=False,norm_hist=False,rug=False,kde=True,ax=axin3,kde_kws={'color':'b'})
sns.distplot((L1[L1<50000]/1000)[1450:1950],bins=25,hist=False,norm_hist=False,rug=False,kde=True,ax=axin3,kde_kws={'color':'r'})
axin3.set_xticks([])
axin3.set_yticks([])

#Subplot 5 - Inverse growthrate
sns.distplot(np.log(numpy.ma.masked_invalid(Time)/86400), hist=True,bins=25,norm_hist=False,rug=False,label='Total',kde=False,ax=ax[4])
ax[4].set_xticks([-2,0,2,4,6,8])
                
ax[4].set_xticklabels(
           [r'$10^{-2}$',r'$10^0$','$10^2$', r'$10^4$', r'$10^6$', r'$10^8$'])
ax[4].set_xlabel('Inverse growth rate (days)')


axin4 = ax[4].inset_axes(
        [2, 550, 6, 250], transform=ax[4].transData)

sns.distplot(np.log(numpy.ma.masked_invalid(Time)/86400)[10:490],bins=25,hist=False,norm_hist=False,rug=False,kde=True,ax=axin4,kde_kws={'color':'b'})
sns.distplot(np.log(numpy.ma.masked_invalid(Time)/86400)[1450:1950],bins=25,hist=False,norm_hist=False,rug=False,kde=True,ax=axin4,kde_kws={'color':'r'})
axin4.set_xticks([])
axin4.set_yticks([])

#plt.savefig('figures/hist_summer_2018.png',transparent=True,dpi=75,bbox_inches='tight')


## Atmospheric derived surface fluxes

In [ ]:
# Q is given in W/m2
Qnet_interp=sms_fluxes.glider_compat(era5_annual.time,time,Qnet) # Interpolated to glider track


To convert freshwater flux from m/s to an equivalent heat flux, first FWF is converted to a buoyancy:

$$
F_b = (E-P)  \times \beta \times g \times S
$$

and then converted to an equivalent heat flux

$$
Q_{FWF} = \frac {F_b \times \rho_0 \times C_p}  {\alpha \times g}
$$


In [ ]:
plt.plot(pd.Series(SA[15,:]).interpolate())

In [ ]:
#  FWF computation
fwf_interp=sms_fluxes.glider_compat(erafw.time,time,fwfms) # Interpolated to glider track
F_b=fwf_interp*beta[10,:]*g*pd.Series(SA[15,:]).interpolate()
Qnet_F=(F_b*(1027*4000)/(alpha[10,:]*g))

In [ ]:
Qsurf = Qnet_interp+Qnet_F

In [ ]:
tau, tau_x, tau_y, wind_dir = sms_fluxes.calc_wind(era5w_subset.u10[:,0],era5w_subset.v10[:,0]) # Wind Stress
tx_interp=pd.Series(sms_fluxes.glider_compat(era5w_subset.time,time,tau_x)).rolling(1).mean() # Interpolated to glider track
ty_interp=pd.Series(sms_fluxes.glider_compat(era5w_subset.time,time,tau_y)).rolling(1).mean()
wind_dir_interp=pd.Series(sms_fluxes.glider_compat(era5w_subset.time,time,wind_dir)).rolling(1).mean()

In [ ]:
print('mean wind stress:',np.nanmean(np.sqrt(tx_interp**2+ty_interp**2)))

In [ ]:
import seaborn as sns
fig,ax=plt.subplots(3,1,figsize=[18,15],sharex=True)
sns.set(font_scale=2)

sns.set_style('white')
sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})


import matplotlib
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.projections import get_projection_class


#heat flux
ax[0].plot(era5_annual.time,era5_annual.msnswrf[:,2,4],label='Q$_{shortwave}$',alpha=0.8,linewidth=3)
ax[0].plot(era5_annual.time,era5_annual.msnlwrf[:,2,4],label='Q$_{longwave}$',alpha=0.8,linewidth=3)
ax[0].plot(era5_annual.time,era5_annual.mslhf[:,2,4],label='Q$_{latent}$',alpha=0.8,linewidth=3)
ax[0].plot(era5_annual.time,era5_annual.msshf[:,2,4],label='Q$_{sensible}$',alpha=0.8,linewidth=3)
ax[0].plot(era5_annual.time,pd.Series(era5_annual.mslhf[:,2,4]+era5_annual.msshf[:,2,4]+era5_annual.msnswrf[:,2,4]+era5_annual.msnlwrf[:,2,4]).rolling(24).mean(),c='k',linewidth=4,label='Q$_{net}$')
ax[0].legend(loc='lower center',framealpha=0.6,ncol=6)
ax[0].axhline(0,c='k',linestyle='--',alpha=0.5)
ax[0].set_ylim(-500,900)
ax[0].set_ylabel('Heat Flux (W m$^{-2}$)')

# Freshwater Flux
ax[1].plot(erafw.time,(erafw.mer+erafw.mtpr)/3600,c='k',linewidth=2)
ax[1].axhline(0,c='k',linestyle='--',alpha=0.5)
ax[1].ticklabel_format(axis='y',style='sci',scilimits=(-7,7))
ax[1].yaxis.major.formatter._useMathText = True
ax[1].set_ylabel('FWF (m s$^{-1}$)')

# Winds
ax[2].plot(time,np.sqrt(tx_interp**2+ty_interp**2),c='k',linewidth=2)
ax[2].axhline(np.nanmean(np.sqrt(tx_interp**2+ty_interp**2)),c='k',linestyle='--',alpha=0.5)
ax[2].set_ylim(0,1)
ax[2].set_ylabel('Wind Stress (N m$^{2}$)')
ax[2].set_xlim(time.iloc[0],time.iloc[-1])
h_fmt= mdates.DateFormatter('%d %b')
ax[2].xaxis.set_major_formatter(h_fmt)

# Wind direction inset
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

axins=inset_axes(ax[2],width="170%", height="50%",
                 axes_class = matplotlib.projections.get_projection_class('polar'),
                 borderpad=2
)

normalise = pd.Series((wind_dir_interp+ 360) % 360).interpolate()   # First normalise winds from -180-180 to 0-360, interpolate over nans

histogram, bins = np.histogram(np.deg2rad(normalise), bins=25)
bin_centers = 0.5*(bins[1:] + bins[:-1])
axins.bar(bin_centers, histogram,label="Wind Orientation",color='lightblue',bottom=0.0,alpha=0.8,edgecolor='tab:blue')
axins.grid(alpha=0.2) 
axins.yaxis.get_major_locator().base.set_params(nbins=5)

axins.tick_params(labelsize=15)

#plt.savefig('figures/sg643_atmospheric_conditions.png',bbox_inches='tight',dpi=75)

In [ ]:
# Mesoscale Ekman Buoyancy Flux

# Overview of mesoscale transects
fig = plt.figure(constrained_layout=False,figsize=[15,10])
gs = gridspec.GridSpec(2, 2, figure=fig,width_ratios=[0.3,0.7])
ax1 = plt.subplot(gs[0,1])
ax2 = plt.subplot(gs[1,1])
ax3 = plt.subplot(gs[0,0])
ax4 = plt.subplot(gs[1,0])

cs1=ax3.scatter(lon[120:290],lat[120:290],c=SA[15,120:290],cmap=cmo.haline,vmin=34.0,vmax=34.2)
xx,yy=np.meshgrid(pd.Series(lat[120:290]).interpolate(),pd.Series(depth).interpolate())
ax1.pcolormesh(xx,yy,by[:,120:290],cmap=cmo.balance,vmin=-5e-3,vmax=-1.5e-3)


ax1.set_ylim(60,10)
ax1.set_xlim(-60.4,-59.25)
ax1.plot(lat[120:290],mld[120:290],c='k')

ax4.scatter(lon[1365:1485],lat[1365:1485],c=SA[15,1365:1485],cmap=cmo.haline,vmin=34.0,vmax=34.2)
xx,yy=np.meshgrid(lat[1365:1485],depth)


cs=ax2.pcolormesh(xx,yy,by[:,1365:1485],cmap=cmo.balance,vmin=-5e-3,vmax=-1.5e-3)
ax2.set_ylim(60,10)
ax2.set_xlim(-60.4,-59.25)
ax2.plot(lat[1365:1485],mld[1365:1485],c='k')

cax = fig.add_axes([0.97, 0.58, 0.03, 0.25])
cax2 = fig.add_axes([0.97, 0.18, 0.03, 0.25])

cbar1=plt.colorbar(cs1,cax=cax)
cbar2=plt.colorbar(cs,cax=cax2)

cbar1.set_label('Salinity (g kg$^{-1}$)')
cbar2.set_label('Buoyancy (m s$^{-2}$)')

cbar2.formatter.set_powerlimits((0, 0))
cbar2.formatter._useMathText = True

ax1.set_xticklabels([])

ax3.set_ylabel('Latitude')
ax4.set_ylabel('Latitude')
ax2.set_xlabel('Latitude')
ax2.set_ylabel('Depth (m)')
ax1.set_ylabel('Depth (m)')

ax4.set_xlabel('Longitude')


#plt.savefig('figures/mesocale_transects.png',bbox_inches='tight',dpi=75)

In [ ]:
# Estimation of Meridional Ekman Velocity and EBF

# Mesoscale Transect 1
i = 130
j= 270

nablaS = (np.nanmean(SA[15,i:i+20])-np.nanmean(SA[15,j:j+20]))/((j-i)*1000)
Ue = 1/(1027*f*mld[i:j+20])*(tx_interp[i:j+20])  # using non-rotated zonal wind stress, comparison with rotated component didn't change the result significantly
Ekmany_t1=Ue*nablaS*beta[10,i:j+20]*g*mld[i:j+20]
Qnet_ekmany_t1=(Ekmany_t1*(1027*4000)/(alpha[10,i:j+20]*g))

# Mesoscale Transect 2

k=1375
l=1495

nablaS = (np.nanmean(SA[15,k:k+20])-np.nanmean(SA[15,l:l+20]))/((l-k)*1000)
Ue = 1/(1027*f*mld[k:l+20])*(tx_interp[k:l+20])
Ekmany_t2=Ue*nablaS*beta[10,k:l+20]*g*mld[k:l+20]
Qnet_ekmany_t2=(Ekmany_t2*(1027*4000)/(alpha[10,k:l+20]*g))


# Figure
plt.figure(figsize=[15,6])
plt.plot(time[i:j+20],Qnet_ekmany_t1)
plt.plot(time[k:l+20],Qnet_ekmany_t2)
plt.axhline(0,c='k',alpha=0.5,linestyle='--')
plt.ylabel('Q$_{ekman}$ (W m$^{-2}$)')
plt.xlabel('Time')

## Submesoscale Heat Flux Calculations 

Submesoscale Ekman Buoyancy Flux defined by Thomas et al 2005

$$
Q_{EBF} = \frac {-b_x \tau _y} {f} \frac {C_p} {\alpha g}
$$

In [ ]:

ebf,ty,angle = sms_fluxes.calc_ebf(pd.Series(bxml).interpolate(),wind_dir_interp[:-1],glider_dir[:],
                          tx_interp[:-1],ty_interp[:-1],f,alpha[10,:-1],cp=4000,g=9.8) 

Submesoscale MLE as in Fox-Kemper et al 2008

$$
Q_{MLE} = 0.06 \frac {b_x^2 H^2} {f} \frac  {C_p \rho _0} {\alpha g}
$$

In [ ]:
# Use 4km rolling mean to equate the heat flux to the mesoscale field

Qmle = sms_fluxes.calc_qmle(pd.Series(bxml[:]).rolling(4).mean().interpolate(),pd.Series(mld[:-1]).interpolate(),alpha[10,:-1],f) # Equivalent Heat flux from mixed layer eddies


### example methodology



In [ ]:
#500: Summer subset
import seaborn as sns
sns.set(font_scale=1.3)
from matplotlib import pyplot, lines

import cartopy
import cartopy.crs as ccrs

import matplotlib
import matplotlib.patches as mpatches
from cartopy.util import add_cyclic_point
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import shapefile   
from mpl_toolkits.axes_grid1.inset_locator import inset_axes ,InsetPosition,mark_inset

import matplotlib.path as mpath
import cmocean.cm as cmo


sns.set_style('white')
sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})

fig = plt.figure(constrained_layout=False,figsize=[15,10])
gs = gridspec.GridSpec(4, 2, figure=fig,width_ratios=[0.4,0.6])
plt.subplots_adjust(wspace=0.3)
projection =ccrs.Mercator()#
data_crs=ccrs.PlateCarree()
ax1 = plt.subplot(gs[:,0],projection=projection)
ax2 = plt.subplot(gs[0,1])
ax3 = plt.subplot(gs[1,1])
ax4 = plt.subplot(gs[2,1])
ax5 = plt.subplot(gs[3,1])

# Select interval
i = 505
j = 540

ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
cs=ax1.scatter(lon[i:j],lat[i:j],c=by[15,i:j],s=80,vmin=-3e-3,vmax=-1e-3,cmap=cmo.balance,transform=data_crs) #buoayncy

cax,kw = matplotlib.colorbar.make_axes(ax1,location='top',pad=0.1,shrink=0.7)
out=fig.colorbar(cs,cax=cax,**kw)
label=out.set_label('Buoyancy (m s$^{-2}$)',size=14,verticalalignment='bottom')

out.formatter.set_powerlimits((0, 0))
out.formatter._useMathText = True
out.update_ticks()

Q=ax1.quiver(np.array(lon[i:j]),np.array(lat[i:j]),np.array(tx_interp[i:j]),np.array(ty_interp[i:j]),alpha=0.5,scale=0.5,transform=data_crs)

qk = ax1.quiverkey(Q, 0.78, 0.90, 0.1, r'$\tau$: 0.1 N.m$^{-2}$', labelpos='N',
                 coordinates='axes',labelcolor='Grey',alpha=0.5)  # quiver key

# Clean up map
ax1.set_extent([-0.25, 0.15, -60.15, -59.9])
gl = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.2, linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlocator = mticker.FixedLocator([-0.3,-0.2, -0.1,0, 0.1,0.2])
gl.ylocator = mticker.FixedLocator([-60.15,-60.1,-60.05, -60, -59.95,-59.9])

gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 10, 'color': 'k'}
gl.xlabel_style = {'color': 'k'}

gl.ylabel_style = {'size': 10, 'color': 'k'}
gl.ylabel_style = {'color': 'k'}

ax1.set_ylabel('Longitude ($^o$E)')
ax1.set_xlabel('Latitude ($^o$N)')


transform = ccrs.PlateCarree()._as_mpl_transform(ax1)

# Add annotations
ax1.annotate('Start', xy=(lon[505],lat[505]),  xycoords=transform,
            xytext=(0.05,-59.98),
            arrowprops=dict(arrowstyle="fancy",
                            fc="0.1", ec="none",
                            connectionstyle="angle3,angleA=0,angleB=-90"))

ax1.annotate('End', xy=(lon[539],lat[539]),  xycoords=transform,
            xytext=(-.23,-59.92),
            arrowprops=dict(arrowstyle="fancy",
                            fc="0.1", ec="none",
                            connectionstyle="angle3,angleA=0,angleB=-90"))

################################################

# Buoyancy

x,y=np.meshgrid(np.array(time[505:540]),depth[:50])
cs=ax2.pcolormesh(x,y,by[:50,505:540],vmin=-3e-3,vmax=-1e-3,cmap=cmo.balance)

ax2.plot(time[505:540],mld[505:540],c='k',lw='2')
ax2.set_ylim(50,5)

# Add annotation to show northward and southward transects

a=pd.to_datetime(np.array(time[505:540]))
x,y = np.array([[(a[0]), (a[14])], [0.24, 0.24]])
line = lines.Line2D(x, y, lw=5., color='k', alpha=0.9)
ax2.add_line(line)
line.set_clip_on(False)

x,y = np.array([[(a[15]), (a[34])], [0.24, 0.24]])
line = lines.Line2D(x, y, lw=5., linestyle='--',color='k', alpha=0.9)
ax2.add_line(line)
line.set_clip_on(False)

ax2.text(a[5],-2,'Southward')
ax2.text(a[19],-2,'Northward')

ax2.text(a[4],15,'Buoyancy',color='k')

################################################
# Rotated winds

ax3.plot(time[505:521],ty[505:521],c='k')
ax3.plot(time[520:540],ty[520:540],c='k',linestyle='--',)
ax3.axhline(0,c='Grey',alpha=0.5,linestyle='--')
ax3.set_ylabel(r'$\tau^y$ (N m$^{-2}$)')

################################################
# Lateral Buoyancy Gradient
ax4.plot(time[505:521],bxml[505:521],c='k',linestyle='-')
ax4.plot(time[520:540],bxml[520:540],c='k',linestyle='--')

ax4.ticklabel_format(axis='y',style='sci',scilimits=(-8,8))
ax4.yaxis.major.formatter._useMathText = True

ax4.ticklabel_format(axis='y', style='sci', scilimits=(-8,-8))
ax4.yaxis.major.formatter._useMathText = True

ax4.axhline(0,c='Grey',alpha=0.5,linestyle='--')
ax4.set_ylabel('b$_x$ (s$^{-1}$)')

################################################
# SMS fluxes

ax5.plot(time[505:521],ebf[505:521],c='tab:blue',linestyle='-',label='$Q_{EBF}$')
ax5.plot(time[520:540],ebf[520:540],c='tab:blue',linestyle='--')

ax5.plot(time[505:521],Qmle[505:521],c='tab:orange',linestyle='-',label='$Q_{MLE}$')
ax5.plot(time[520:540],Qmle[520:540],c='tab:orange',linestyle='--')

#ax5.set_xlim(np.array(time[505]),np.array(time[539]))
ax5.axhline(0,c='Grey',alpha=0.5,linestyle='--')

ax5.legend(loc='best',framealpha=0.5)
ax5.set_ylabel('Equivalent Heat Flux (W m$^{-2}$)')


# Clean up xticks

h_fmt= mdates.DateFormatter('%a %d')
ax5.xaxis.set_major_formatter(h_fmt)
days = mdates.DayLocator( interval=1, tz=None)
ax5.xaxis.set_major_locator(days)
hours = mdates.HourLocator(byhour=[0,6,12,18,24], interval=1, tz=None)
ax5.xaxis.set_minor_locator(hours)
ax5.set_xlabel('January 2019')




ax5.set_xlim(time[505],time[539])

ax2.set_xlim(time[505],time[539])
ax3.set_xlim(time[505],time[539])
ax4.set_xlim(time[505],time[539])

ax2.xaxis.set_major_locator(days)
ax2.xaxis.set_minor_locator(hours)
ax3.xaxis.set_major_locator(days)
ax3.xaxis.set_minor_locator(hours)
ax4.xaxis.set_major_locator(days)
ax4.xaxis.set_minor_locator(hours)


ax2.set_xticklabels([])
ax3.set_xticklabels([])
ax4.set_xticklabels([])

ax5.set_ylabel('Depth (m)')

#plt.savefig('figures/sms_summer_2018_subset_v3.png',transparent=True,dpi=75,bbox_inches='tight')



### Total

In [ ]:
# Plot all
fig,ax=plt.subplots(3,1,figsize=[18,15],sharex=True)
sns.set(font_scale=2)


sns.set_style('white')

sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})


from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.projections import get_projection_class

#heat flux
Qsurf = pd.Series(Qnet_F).interpolate() + pd.Series(Qnet_interp).interpolate()

ax[0].plot(time,pd.Series(Qsurf).rolling(24).mean(),label='Q$_{surf}$',c='tab:red',alpha=0.8,linewidth=2)
ax[0].plot(time,pd.Series(Qnet_interp).rolling(24).mean(),label='Q$_{net}$',c='k',alpha=0.8,linewidth=2)#ax[0].plot(time, Qfwf_interp,label='Q$_{latent}$',alpha=0.8,linewidth=3)
ax[0].plot(time,pd.Series(Qnet_F),label='Q$_{FWF}$',alpha=0.8,linewidth=2,c='blue')

ax[0].set_ylabel('W m$^{-2}$')

ax[0].legend(ncol=3)
ax[0].axhline(0,linestyle='--',c='Grey',alpha=0.8)

# FWF
ax[1].plot(time.iloc[:-1],pd.Series(ebf).rolling(10).mean().interpolate(),c='tab:blue',linewidth=2,label='Q$_{EBF}$')
ax[1].plot(time.iloc[:-1],Qmle,c='tab:orange',linewidth=2,label='Q$_{MLE}$')


ax[1].axhline(0,c='k',linestyle='--',alpha=0.5)
ax[1].set_ylim(-200,200)
ax[1].set_ylabel('W m$^{-2}$')

Qtot=pd.Series(pd.Series(ebf).rolling(10).mean().interpolate()+Qmle+pd.Series(Qsurf).rolling(24).mean()).interpolate()
ax[1].legend()
ax[1].axhline(0,linestyle='--',c='Grey',alpha=0.8)

# SMS plus atmospheric
ax[2].plot(time,pd.Series(Qsurf).rolling(24).mean(),c='r',linewidth=3,label='Q$_{surf}$')


ax[2].plot(time,Qtot,c='k',linewidth=1,label='Q$_{tot}$')
ax[2].set_xlim(time.iloc[0],time.iloc[-1])
ax[2].legend()
ax[2].set_ylabel('W m$^{-2}$')

ax[2].axhline(0,linestyle='--',c='Grey',alpha=0.8)

a=pd.to_datetime(np.array(time[1954:-1]))
x,y = np.array([[(a[0]), (a[-1])], [320, 320]])
line = lines.Line2D(x, y, lw=5., color='k', alpha=0.9)
ax[0].add_line(line)
line.set_clip_on(False)


ax[0].text(a[5],330,'Autumn',fontsize=15)


h_fmt= mdates.DateFormatter('%d %b')
ax[2].xaxis.set_major_formatter(h_fmt)

#plt.savefig('/Users/isabelle/Documents/src/roammiz/figures/sg643_sms_fluxes_all.png',bbox_inches='tight',dpi=75)



### bx and ebf underestimation analysis

Assume front is oriented north south such that direction of geostrophic shear is east west.   
For glider to sample the front perpendicularly, the glider should be angled north south to capture the full horizontal gradient(i.e. 0). Assume that bx is computed over 1000m = 1e-7. Use all the acute angles of the glider, we can compute the theoretical observed distance across the front and from this estimate how much the glider is underestimating the lateral gradients. 


In [ ]:
o=1000
theta = np.abs(glider_dir)
theta_allpossibilities = np.linspace(1,90,len(glider_dir))
bactual=1e-4/1000 # if glider is perpendicular to front (equivalent to 0degrees here)
bobs=[]
for i in range(len(theta[:])):
    if theta[i]<90:
        h=o/np.cos(np.deg2rad(theta[i]))
        bobs.append(1e-4/h)
    else:
        theta2 = 180-theta[i]
        h=o/np.cos(np.deg2rad(theta2))
        bobs.append(1e-4/h)

ballposs=[]
for i in range(len(theta_allpossibilities[:])):

        h=o/np.sin(np.deg2rad(theta_allpossibilities[i]))
        ballposs.append(1e-4/h)


In [ ]:
allpossiblewindangles=np.linspace(-180,180,len(glider_dir))
windangle=np.ones(len(glider_dir))*0
perpAngle=np.ones(len(glider_dir))*180
actualwindangles=wind_dir_interp[:-1]
windsorted=(actualwindangles)

txt=np.ones(len(glider_dir))*0.1
tyt=np.ones(len(glider_dir))*0.1

ebf_actual,_=sms_fluxes.calc_ebf(1e-7,np.abs(actualwindangles),np.ones(len(glider_dir))*180,txt,tyt,f,5e-5,cp=4000,g=9.8)

#ebf_actual,_=sms_fluxes.calc_ebf(1e-7,[270,270],[90,90],
 #                         [0.1,0.1],[0.1,0.1],f,5e-5,cp=4000,g=9.8)
ebf_obs,_=sms_fluxes.calc_ebf(1e-7,np.abs(actualwindangles),np.abs((glider_dir)),
                          txt,tyt,f,5e-5,cp=4000,g=9.8)

In [ ]:
import seaborn as sns
sns.distplot(np.ma.masked_invalid((ebf_obs[:]))/ebf_actual)
#plt.xlim(-10,10)

In [ ]:
plt.plot(np.sort(ebf_obs))
plt.plot(np.sort(ebf_actual))

In [ ]:
import sklearn.metrics
import math
mse = sklearn.metrics.mean_squared_error( np.sort(bobs),np.ones(len(bobs))*1e-7)
print(mse)
rmse = math.sqrt(mse)
print(rmse)
print(np.nanmean(rmse/1e-7))

In [ ]:
mse = sklearn.metrics.mean_squared_error((ebf_obs),(ebf_actual))
#rint(mse)
print(rmse)
print(np.sort(ebf_actual))
print(np.sort(ebf_obs))
print(np.nanmean(rmse/np.nanmean(ebf_actual)))
#1-0.43

## Spectra Analysis

In [ ]:
from spectrum import *
from JD_multitaper import JD_spectra
from scipy.fftpack import fft
from scipy import signal
from JD_multitaper import JD_space_spectra, plot_slope, plot_ref_slope
from scipy import stats

def detrend(h):
    n = len(h)
    t = np.arange(n)
    p = np.polyfit(t, h, 1)
    h_detrended = h - np.polyval(p, t)
    return h_detrended
    
def quadwin(n):
    """
    Quadratic (or "Welch") window
    """
    t = np.arange(n)
    win = 1 - ((t - 0.5 * n) / (0.5 * n)) ** 2
    return win

In [ ]:
# Run initial spectral analysis

fig,ax=plt.subplots(figsize=[7,8])
plt.rcParams.update({'font.size': 16})

#####################################################

#### All Summer
rho1=pd.Series(np.nanmean(rho[10:30,10:],axis=0))  # Select mean of 10-30m
rho1=rho1.interpolate()
n = len(rho1[10:])
h_detrended = detrend(rho1[10:]) # detrend
winweights = quadwin(n) # filter
h_win_r = h_detrended * winweights

[Px_r3,F_r3,Pxc,hh]=JD_space_spectra(h_win_r, dl=1,
        ax=ax,
        f=1, nw=3, unit='(kg m$^{-3}$)$^2$')

ref_slopes=[2,3]
plt.xlim(1e-2,0)
plt.ylim(1e-9,1e-1)
print('all',plot_slope(Px_r3,F_r3,ax=ax,fmin=1/50,fmax=1/3))
barmax=Pxc[0,0]*0.5e-8
barmin=Pxc[1,0]*0.5e-8
f=1/70
ax.loglog([f, f],[barmin, barmax],color='k',marker='_')

#####################################################



#### Early Summer
rho3=pd.Series(np.nanmean(rho[10:30,10:490],axis=0))    # Select mean of 10-30m
rho3=rho3.interpolate()
n = len(rho3[10:]) 
h_detrended = detrend(rho3[10:])   # detrend 
winweights = quadwin(n)   # filter with a welch window
h_win_r = h_detrended * winweights

[Px_r,F_r,Pxc,hh]=JD_space_spectra(h_win_r, dl=1,
        ax=ax,
        f=1, nw=3, unit='(kg m$^{-3}$)$^2$')

ref_slopes=[2,3]
plt.xlim(1e-2,0)
plt.ylim(1e-9,1e-1)
print('earrrly',plot_slope(Px_r,F_r,ax=ax,fmin=1/50,fmax=1/3))
barmax=Pxc[0,0]*0.5e-8
barmin=Pxc[1,0]*0.5e-8
f=1/70
ax.loglog([f, f],[barmin, barmax],color='k',marker='_')

#####################################################




#### Late Summer
rho2=pd.Series(np.nanmean(rho[10:30,1467:1957],axis=0))
rho2=rho2.interpolate()
n = len(rho2[10:])
h_detrended = detrend(rho2[10:])
winweights = quadwin(n)
h_win_r = h_detrended * winweights

[Px_r2,F_r2,Pxc,hh]=JD_space_spectra(h_win_r, dl=1,
        ax=ax,
        f=1, nw=3, unit='(kg.m$^{-3}$)$^2$')

ref_slopes=[2,3]
print('late',plot_slope(Px_r2,F_r2,ax=ax,fmin=1/50,fmax=1/3))


print(plot_slope(Px_r,F_r,ax=ax,fmin=1/50,fmax=1/3))
barmax=Pxc[0,0]*0.5e-8
barmin=Pxc[1,0]*0.5e-8
f=1/70
ax.loglog([f, f],[barmin, barmax],color='k',marker='_')
#plot_ref_slope(fmin=5e-1,fmax=10**-2,f=10**-4,ax=ax,col='Gray',kvec=ref_slopes)
plt.xlim(1e-2,5e-1)
plt.ylim(1e-9,1e-1)
print(plot_slope(Px_r3,F_r3,ax=ax,fmin=1/50,fmax=1/3))


In [ ]:
# Sensitivity analysis of the choice of slope for early summer

a1=(plot_slope(Px_r,F_r,ax=ax,fmin=1/50,fmax=1/1)[0])
a2=(plot_slope(Px_r,F_r,ax=ax,fmin=1/50,fmax=1/4)[0])
a3=(plot_slope(Px_r,F_r,ax=ax,fmin=1/50,fmax=1/5)[0])
a4=(plot_slope(Px_r,F_r,ax=ax,fmin=1/50,fmax=1/7)[0])
a5=(plot_slope(Px_r,F_r,ax=ax,fmin=1/60,fmax=1/1)[0])
a6=(plot_slope(Px_r,F_r,ax=ax,fmin=1/70,fmax=1/1)[0])
a7=(plot_slope(Px_r,F_r,ax=ax,fmin=1/80,fmax=1/1)[0])
a8=(plot_slope(Px_r,F_r,ax=ax,fmin=1/90,fmax=1/1)[0])

slope1 = np.array([a1,a2,a3,a4,a5,a6,a7,a8])

means1 = np.nanmean(slope1)

print('mean slope early summer:',np.nanmean(slope1))
print('standard deviation slope early summer:',np.nanstd(slope1))

In [ ]:
# Sensitivity analysis of the choice of slope for late summer

b1=(plot_slope(Px_r2,F_r2,ax=ax,fmin=1/50,fmax=1/1)[0])
b2=(plot_slope(Px_r2,F_r2,ax=ax,fmin=1/50,fmax=1/4)[0])
b3=(plot_slope(Px_r2,F_r2,ax=ax,fmin=1/50,fmax=1/5)[0])
b4=(plot_slope(Px_r2,F_r2,ax=ax,fmin=1/50,fmax=1/7)[0])
b5=(plot_slope(Px_r2,F_r2,ax=ax,fmin=1/60,fmax=1/1)[0])
b6=(plot_slope(Px_r2,F_r2,ax=ax,fmin=1/70,fmax=1/1)[0])
b7=(plot_slope(Px_r2,F_r2,ax=ax,fmin=1/80,fmax=1/1)[0])
b8=(plot_slope(Px_r2,F_r2,ax=ax,fmin=1/90,fmax=1/1)[0])

slopes2 = (np.array([b1,b2,b3,b4,b5,b6,b7,b8]))

means2 = np.nanmean(slopes2)

print('mean slope late summer:',np.nanmean(slopes2))
print('standard deviation slope late summer:',np.nanstd(slopes2))

In [ ]:
# Hack on slope functions to replace with the mean slopes as computed above


def plot_slope1(Px,F,ax,fmin=None,fmax=None,col='k',lin='--',lw=2,offy=1):

    """ This function draws the slope of the spectrum
    Arguments:
    Px=the vector of spectrum
    F=the vector of frequency
    ax=the axes in which the slopes are plotted
    
    Optional arguments:
    fmin=None (if fmin=None, takes the min of F): frequency minimum where the slope is computed
    fmax=None (if fmax=None, takes the max of F): frequency maximum where the slope is computed
    col='k': color of the slope
    lin='--': linestyle of the slope
    lw=2: linewidth of the slope
    offy=1: y offset of the slope
    
    Outputs:
    The value of the slope

    Author: Nicolas Barrier
    """

    if fmin==None:
        fmin=F.min()
    if fmax==None:
        fmax=F.max()

    i=np.nonzero((F>=fmin)&(F<=fmax)&(F!=0))[0]
    fout=F[i]
    pxout=Px[i]

    y=np.log(pxout)
    x=np.log(fout)
    from scipy import stats
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

    confidence_interval = 2.58*std_err #99%
    
    p=np.polyfit(x,y,1)
    
    
   # trend=np.exp(p[0]*x+p[1]+offy)
    trend=np.exp(means2*x+p[1]+offy)


    
    ax.loglog(fout,trend,color=col,linestyle=lin,lw=lw)
    
    
def plot_slope2(Px,F,ax,fmin=None,fmax=None,col='k',lin='--',lw=2,offy=1):

    """ This function draws the slope of the spectrum
    Arguments:
    Px=the vector of spectrum
    F=the vector of frequency
    ax=the axes in which the slopes are plotted
    
    Optional arguments:
    fmin=None (if fmin=None, takes the min of F): frequency minimum where the slope is computed
    fmax=None (if fmax=None, takes the max of F): frequency maximum where the slope is computed
    col='k': color of the slope
    lin='--': linestyle of the slope
    lw=2: linewidth of the slope
    offy=1: y offset of the slope
    
    Outputs:
    The value of the slope

    Author: Nicolas Barrier
    """

    if fmin==None:
        fmin=F.min()
    if fmax==None:
        fmax=F.max()

    i=np.nonzero((F>=fmin)&(F<=fmax)&(F!=0))[0]
    fout=F[i]
    pxout=Px[i]

    y=np.log(pxout)
    x=np.log(fout)
    from scipy import stats
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

    confidence_interval = 2.58*std_err #99%
    
    p=np.polyfit(x,y,1)
    
    
   # trend=np.exp(p[0]*x+p[1]+offy)
    trend=np.exp(means1*x+p[1]+offy)


    
    ax.loglog(fout,trend,color=col,linestyle=lin,lw=lw)
    


In [ ]:
# The formatting of this is still a bit of a hack, but it works :P 

fig,ax=plt.subplots(1,1,figsize=[10,10])
ax.loglog(F_r3,Px_r3,c='Grey',alpha=0.5,linewidth=3,label='Total')
ax.loglog(F_r,Px_r,c='tab:blue',linewidth=4,label='Early Summer')


ax.loglog(F_r2,Px_r2,c='tab:red',linewidth=4,label='Late Summer')
plot_ref_slope(fmin=5e-1,fmax=10**-2,f=10**-4,ax=ax,col='Gray',kvec=ref_slopes)



plt.legend(fontsize=16)
ax.set_xlim(8e-3,5e-1)
ax.set_ylim(1.5e-8,1e-1)
barmax=Pxc[0,0]*1e-7
barmin=Pxc[1,0]*1e-7
f=1/70
ax.loglog([f, f],[barmin, barmax],color='k',marker='_')

#############

plot_slope1(Px_r2,F_r2,ax=ax,fmin=1/50,fmax=1/3,col='tab:red')

############

plot_slope2(Px_r,F_r,ax=ax,fmin=1/50,fmax=1/3,col='tab:blue')

ax.set_xlabel('Wavenumber (cpkm)',fontsize=16)
ax.set_ylabel('Power Spectrum Density (kg m$^{-3}$)$^2$ cpkm$^{-1}$',fontsize=16)


ax.annotate('$k^{-2}$', xy=(1e-1,1.9e-3),  xycoords='data',
            xytext=(1e-1,1e-3),
           )

ax.annotate('$k^{-3}$', xy=(1e-1,2e-2),  xycoords='data',
            xytext=(1e-1,2e-2),
           )

#plt.savefig('figures/power_spectra.png',transparent=True,dpi=150,bbox_inches='tight')
#

## Spice analysis

We suspect little vertical exchange across the thermocline in this region.  
This is associated with increased stratification by sea ice melt water. 
i.e. MLE are confined to the ML.  
Here, this hypothesis is tested

In [ ]:
# First call regridded data along isopycnals
ds_iso = xr.open_dataset('data/sg643/sg643_grid_density_surfaces.nc')


In [ ]:
ds_iso

In [ ]:
spice_iso=gsw.spiciness0(ds_iso.salinity,ds_iso.sea_water_temperature)  # Compute spiciness along isopycnals

In [ ]:
# Look at salinity by density surfaces
xx,yy=np.meshgrid(ds_iso.distance/1000,ds_iso.density)
plt.figure(figsize=[15,5])
plt.pcolormesh(xx,yy,ds_iso.salinity,vmin=33.9,vmax=34.7,cmap=cmo.haline) # blues are the ML
plt.ylim(ds_iso.density.max(),ds_iso.density.min())
plt.colorbar()

In [ ]:
#Spiciness
xx,yy=np.meshgrid(ds_iso.distance/1000,ds_iso.density)
plt.figure(figsize=[15,5])
plt.pcolormesh(xx,yy,spice_iso) # blues are the ML
plt.ylim(ds_iso.density.max(),ds_iso.density.min())
plt.colorbar()

In [ ]:
spice_ML = pd.Series(np.nanmean(spice_iso[25:30,:],axis=0))
spice_deeper = pd.Series(np.nanmean(spice_iso[55:60,:],axis=0))

In [ ]:
# Visualise on two isopycnals (surface and at depth)
fig,ax=plt.subplots(2,1,figsize=[12,6],sharex=True)

ax[0].set_title(np.nanmean(ds_iso['density'][25:30]))
ax[0].plot(time,pd.Series(np.nanmean(spice_iso[25:30,:],axis=0)))
ax[1].set_title(np.nanmean(ds_iso['density'][55:60]))
ax[1].plot(time,pd.Series(np.nanmean(spice_iso[55:60,:],axis=0)))

In [ ]:
# Correlation analysis

In [ ]:
plt.scatter(spice_ML[:],spice_deeper[:])  # There is some structure but not in the whole timeseries

In [ ]:
from corr_JD import covariance,covlag, correlation
covariance(spice_ML[1:],spice_deeper[1:])

In [ ]:
def covlag(x,y,maxlag):

    """ Compute the lead-lag covariance between two time series, given a maximum lag.
    
    Arguments:
    x: 1D array (dominates at positive lags)
    y: 1D array
    maxlag: maximum number of lags (integer)
    
    Outputs:
    tuple(ccv,lag), with ccv the cross-covariance and lag the lag vectors
    
    Author: Julie Deshayes
    """

    x=np.ma.array(x,mask=x!=x)
    
    y=np.ma.array(y,mask=y!=y)

    n=len(x)
    xleady=np.zeros(maxlag+1)
    yleadx=np.zeros(maxlag+1)
    
    lag=np.arange(0,maxlag+1,1)

    for l in lag:
        
        xint=x[l:]
        yint=y[0:n-l]
        iok=np.nonzero((xint.mask==False)&(yint.mask==False))[0]
        if len(iok)>0:
            yleadx[l]=covariance(xint[iok],yint[iok])
  
        yint=y[l:]
        xint=x[0:n-l]
        iok=np.nonzero((xint.mask==False)&(yint.mask==False))[0]
        if len(iok)>0:
            xleady[l]=covariance(xint[iok],yint[iok])
  
    ccv=np.empty(2*maxlag+1)
    ccv[0:maxlag]=yleadx[1:][::-1]
    ccv[maxlag:]=xleady
    ccv=np.ma.masked_where(ccv==0,ccv)

    lag=np.arange(-maxlag,maxlag+1,1)
    return ccv,lag

In [ ]:
def corlag(x,y,maxlag):

    """ Compute the lead-lag correlation between two time series, given a maximum lag.
    
    Arguments:
    x: 1D array (dominates at positive lags)
    y: 1D array
    maxlag: maximum number of lags (integer)
    
    Outputs:
    tuple(ccr,lag), with ccr the cross-correlation and lag the lag vectors
    
    Author: Nicolas Barrier
    """

    x=np.ma.array(x,mask=x!=x)
    
    y=np.ma.array(y,mask=y!=y)

    n=len(x)
    xleady=np.zeros(maxlag+1)
    yleadx=np.zeros(maxlag+1)
   
    #eval(['print'+y+' leads for negative lags'])
    
    lag=np.arange(0,maxlag+1,1)

    for l in lag:
        
        xint=x[l:]
        yint=y[0:n-l]
        iok=np.nonzero((xint.mask==False)&(yint.mask==False))[0]
        if len(iok)>0:
            yleadx[l]=correlation(xint[iok],yint[iok])
  
        yint=y[l:]
        xint=x[0:n-l]
        iok=np.nonzero((xint.mask==False)&(yint.mask==False))[0]
        if len(iok)>0:
            xleady[l]=correlation(xint[iok],yint[iok])[0,1]
  
    ccr=np.empty(2*maxlag+1)
    ccr[0:maxlag]=yleadx[1:][::-1]
    ccr[maxlag:]=xleady
    ccr=np.ma.masked_where(ccr==0,ccr)

    lag=np.arange(-maxlag,maxlag+1,1)
    return ccr,lag

In [ ]:
spice_deeper = pd.Series(np.nanmean(spice_iso[55:60,:],axis=0))

In [ ]:
ccv,lag=covlag(spice_ML[1:1000],spice_deeper[1:1000],maxlag=30)

In [ ]:
plt.plot(lag,ccv)
plt.ylabel('Cross covariance')
plt.xlabel('Lags (km)')